# 180830 Make Negative Mask

In [2]:
import tensorflow as tf
import numpy as np
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
device_config = tf.ConfigProto()
device_config.gpu_options.allow_growth = True

In [220]:
batch_size = 5
embed_dim = 10

In [227]:
def make_negative_mask(distances, num_negative_samples, method="random"):
    cur_batch_length = tf.shape(distances)[0]
    if method == "random":
        topk = tf.contrib.framework.sort(tf.nn.top_k(tf.random_uniform([cur_batch_length, cur_batch_length]), k=num_negative_samples).indices, axis=1)
        rows = tf.transpose(tf.reshape(tf.tile(tf.range(cur_batch_length), [num_negative_samples+1]), [num_negative_samples+1, cur_batch_length]))
        indices = tf.to_int64(tf.reshape(tf.concat([tf.expand_dims(rows, -1), tf.expand_dims(topk, -1)], axis=2), [(num_negative_samples+1)*cur_batch_length, 2]))
        mask = tf.sparse_to_dense(sparse_indices=indices, 
                                  output_shape=[tf.to_int64(cur_batch_length), tf.to_int64(cur_batch_length)], 
                                  sparse_values=tf.ones([((num_negative_samples+1)*cur_batch_length)], 1))
        
        # drop positive
        mask = tf.multiply(mask, (1- tf.eye(cur_batch_length)))
        
    elif method == "hard":
        topk = tf.contrib.framework.sort(tf.nn.top_k(distances, k=num_negative_samples+1).indices, axis=1)
        rows = tf.transpose(tf.reshape(tf.tile(tf.range(cur_batch_length), [num_negative_samples+1]), [num_negative_samples+1, cur_batch_length]))
        indices = tf.to_int64(tf.reshape(tf.concat([tf.expand_dims(rows, -1), tf.expand_dims(topk, -1)], axis=2), [(num_negative_samples+1)*cur_batch_length, 2]))
        mask = tf.sparse_to_dense(sparse_indices=indices, 
                                  output_shape=[tf.to_int64(cur_batch_length), tf.to_int64(cur_batch_length)], 
                                  sparse_values=tf.ones([((num_negative_samples+1)*cur_batch_length)], 1))
        # drop positive
        mask = tf.multiply(mask, (1- tf.eye(cur_batch_length)))
        
    elif method == "weighted":
        weight = tf.map_fn(lambda x: get_distance_weight(x, batch_size), tf.to_float(distances))
        mask = weight
#         mask = tf.to_int32(tf.contrib.framework.sort(tf.expand_dims(tf.multinomial(weight, num_negative_samples+1), -1), axis=1))
#         weighted_samples_indices = tf.to_int32(tf.contrib.framework.sort(tf.expand_dims(tf.multinomial(weight, num_negative_samples+1), -1), axis=1))
#         row_indices = tf.expand_dims(tf.transpose(tf.reshape(tf.tile(tf.range(0, batch_size, 1), [num_negative_samples+1]), [num_negative_samples+1, batch_size])), -1)
#         mask_indices = tf.to_int64(tf.squeeze(tf.reshape(tf.concat([row_indices, weighted_samples_indices], 2), [(num_negative_samples+1)*batch_size,1,2])))
#         mask_sparse = tf.SparseTensor(mask_indices, [1]*((num_negative_samples+1)*batch_size), [batch_size,batch_size])
#         mask = tf.sparse_tensor_to_dense(mask_sparse)
#         drop_positive = tf.to_int32(tf.subtract(tf.ones([batch_size, batch_size]), tf.eye(batch_size)))
#         mask = tf.multiply(mask, drop_positive)
        
    return mask

In [228]:
graph = tf.Graph()
with graph.as_default():
    source = np.random.random(size=[batch_size, embed_dim])
    encoding_queries = tf.convert_to_tensor(source)
    encoding_replies = tf.convert_to_tensor(source)
    distances = tf.matmul(encoding_queries, encoding_replies, transpose_b=True)
    num_negative_samples = tf.constant(2)
    negative_mask = make_negative_mask(distances, num_negative_samples, method="hard")

In [229]:
sess = tf.Session(config=device_config, graph=graph)

In [231]:
sess.run(negative_mask)

array([[0., 1., 0., 0., 1.],
       [0., 0., 0., 1., 1.],
       [1., 0., 0., 0., 1.],
       [1., 1., 0., 0., 0.],
       [1., 1., 0., 0., 0.]], dtype=float32)

In [224]:
sess.run(topk)

array([[0, 1, 4],
       [1, 3, 4],
       [1, 2, 3],
       [0, 1, 3],
       [0, 1, 4]], dtype=int32)

In [225]:
sess.run(indices)

array([[0, 0],
       [0, 1],
       [0, 4],
       [1, 1],
       [1, 3],
       [1, 4],
       [2, 1],
       [2, 2],
       [2, 3],
       [3, 0],
       [3, 1],
       [3, 3],
       [4, 0],
       [4, 1],
       [4, 4]])

In [226]:
sess.run(mask)

array([[1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 1.],
       [0., 1., 1., 1., 0.],
       [1., 1., 0., 1., 0.],
       [1., 1., 0., 0., 1.]], dtype=float32)

In [90]:
top_k = tf.contrib.framework.sort(tf.expand_dims(tf.nn.top_k(-distances, k=num_negative_samples+1).indices, -1), axis=1)
row_indices = tf.expand_dims(tf.transpose(tf.reshape(tf.tile(tf.range(0, batch_size, 1), [num_negative_samples+1]), [num_negative_samples+1, batch_size])), -1)
mask_indices = tf.to_int64(tf.squeeze(tf.reshape(tf.concat([row_indices, top_k], 2), [(num_negative_samples+1)*batch_size,1,2])))
mask_sparse = tf.SparseTensor(mask_indices, [1]*((num_negative_samples+1)*batch_size), [batch_size,batch_size])
mask = tf.sparse_tensor_to_dense(mask_sparse)
drop_positive = tf.to_int32(tf.subtract(tf.ones([batch_size, batch_size]), tf.eye(batch_size)))
mask = tf.multiply(mask, drop_positive)

Exception ignored in: <bound method ScopedTFGraph.__del__ of <tensorflow.python.framework.c_api_util.ScopedTFGraph object at 0x7f5a2c0cd1d0>>
Traceback (most recent call last):
  File "/home/angrypark/angryenv/lib/python3.5/site-packages/tensorflow/python/framework/c_api_util.py", line 48, in __del__
    def __del__(self):
KeyboardInterrupt: 


ValueError: Tensor("Neg_1:0", shape=(5, ?, 1), dtype=int32) must be from the same graph as Tensor("sort/transposition:0", shape=(3,), dtype=int64).

In [14]:
sess.run(distances)

array([[2.29511798, 0.67700572, 1.34751248, 1.27330097, 0.98965927,
        0.73284263, 1.279159  , 1.71423943, 1.81772648, 1.69682994],
       [1.98197293, 0.54303031, 1.65039725, 1.42866775, 0.71762133,
        0.73568608, 0.75333403, 1.47168005, 1.42029444, 1.48372218],
       [2.1536218 , 0.75326856, 1.18513704, 1.52605878, 0.56687777,
        0.45480422, 0.63859215, 1.72974552, 1.81409552, 1.34178155],
       [2.64062429, 0.80540714, 1.57387152, 1.65055524, 0.91815799,
        0.7693199 , 1.18541877, 1.98628414, 2.07976949, 1.83032612],
       [2.30033065, 0.69046371, 1.95800522, 1.53890671, 1.23578142,
        0.85464199, 1.08251009, 1.8481986 , 1.80236427, 1.91579579],
       [2.8860319 , 0.97263702, 2.2314912 , 1.78506514, 1.60017632,
        0.98383685, 1.39318532, 2.55018975, 2.5882097 , 2.31123511],
       [1.81154412, 0.61226773, 1.16854531, 1.04234277, 0.71379987,
        0.58758445, 0.82177858, 1.6178264 , 1.71351778, 1.22956619],
       [1.48346709, 0.54555088, 1.2815947